In [1]:
!pip install pyspark

In [2]:
import pyspark
import os
from pyspark.sql import SparkSession
from pyspark.sql import types
from pyspark.sql import functions as F

In [3]:
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.3.1,org.apache.spark:spark-avro_2.12:3.3.1 pyspark-shell'

In [4]:
spark = SparkSession \
    .builder \
    .master("local[*]") \
    .appName("GreenTripsConsumer") \
    .getOrCreate()

24/04/01 16:32:37 WARN Utils: Your hostname, Oluseguns-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.109 instead (on interface en0)
24/04/01 16:32:37 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /Users/iamraphson/.ivy2/cache
The jars for the packages stored in: /Users/iamraphson/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-5dc02dae-fe39-4e54-a29c-5420a4f2c399;1.0
	confs: [default]


:: loading settings :: url = jar:file:/opt/homebrew/Cellar/apache-spark/3.5.1/libexec/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.3.1 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.3.1 in central
	found org.apache.kafka#kafka-clients;2.8.1 in central
	found org.lz4#lz4-java;1.8.0 in central
	found org.xerial.snappy#snappy-java;1.1.8.4 in central
	found org.slf4j#slf4j-api;1.7.32 in central
	found org.apache.hadoop#hadoop-client-runtime;3.3.2 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.hadoop#hadoop-client-api;3.3.2 in central
	found commons-logging#commons-logging;1.1.3 in central
	found com.google.code.findbugs#jsr305;3.0.0 in central
	found org.apache.commons#commons-pool2;2.11.1 in central
	found org.apache.spark#spark-avro_2.12;3.3.1 in central
	found org.tukaani#xz;1.8 in central
:: resolution report :: resolve 270ms :: artifacts dl 7ms
	:: modules in use:
	com.google.code.findbugs#jsr305;3.0.0 from central in [default]
	commons-logging#commons-logging;1.1.3 from central in [default]
	org.ap

In [5]:
green_stream = spark \
    .readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "localhost:9092") \
    .option("subscribe", "green-trips") \
    .option("startingOffsets", "earliest") \
    .load()

In [6]:
schema = types.StructType() \
    .add("lpep_pickup_datetime", types.StringType()) \
    .add("lpep_dropoff_datetime", types.StringType()) \
    .add("PULocationID", types.IntegerType()) \
    .add("DOLocationID", types.IntegerType()) \
    .add("passenger_count", types.DoubleType()) \
    .add("trip_distance", types.DoubleType()) \
    .add("tip_amount", types.DoubleType())


In [7]:
green_stream = green_stream \
    .select(F.from_json(F.col("value").cast('STRING'), schema).alias("data")) \
    .select("data.*") \
    .withColumn("timestamp", F.current_timestamp())
  

In [8]:
popular_destinations = green_stream \
    .groupby(F.window(F.col("timestamp"), "5 minutes"), "DOLocationID") \
    .count() \
    .orderBy(F.desc("count"))

In [10]:
query = popular_destinations \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .option("truncate", "false") \
    .start()

24/04/01 16:39:32 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /private/var/folders/wm/00zdb_c90lj1zjg999tvvklh0000gn/T/temporary-26bba764-98f1-4fc9-864f-66e1ab1a86ad. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
24/04/01 16:39:32 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
24/04/01 16:39:32 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
24/04/01 16:39:32 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
24/04/01 16:39:32 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
24/04/01 16:39:32 WARN AdminClientConfig: The configuration '

-------------------------------------------
Batch: 0
-------------------------------------------
+------------------------------------------+------------+-----+
|window                                    |DOLocationID|count|
+------------------------------------------+------------+-----+
|{2024-04-01 16:35:00, 2024-04-01 16:40:00}|74          |53223|
|{2024-04-01 16:35:00, 2024-04-01 16:40:00}|42          |47826|
|{2024-04-01 16:35:00, 2024-04-01 16:40:00}|41          |42183|
|{2024-04-01 16:35:00, 2024-04-01 16:40:00}|75          |38520|
|{2024-04-01 16:35:00, 2024-04-01 16:40:00}|129         |35790|
|{2024-04-01 16:35:00, 2024-04-01 16:40:00}|7           |34599|
|{2024-04-01 16:35:00, 2024-04-01 16:40:00}|166         |32535|
|{2024-04-01 16:35:00, 2024-04-01 16:40:00}|236         |23739|
|{2024-04-01 16:35:00, 2024-04-01 16:40:00}|223         |22626|
|{2024-04-01 16:35:00, 2024-04-01 16:40:00}|238         |21954|
|{2024-04-01 16:35:00, 2024-04-01 16:40:00}|82          |21876|
|{2024-

In [11]:
query.awaitTermination()

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/opt/homebrew/Cellar/apache-spark/3.5.1/libexec/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/homebrew/Cellar/apache-spark/3.5.1/libexec/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
                          ^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/iamraphson/.pyenv/versions/3.11.0/lib/python3.11/socket.py", line 705, in readinto
    return self._sock.recv_into(b)
           ^^^^^^^^^^^^^^^^^^^^^^^
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
spark.stop()